In [1]:
from datasets import load_dataset


ds_legalRAG = load_dataset("tuananh18/Eval-RAG-Vietnamese", 'legal-data')


data = ds_legalRAG['train'].shuffle(seed=42).select(range(2))


for row in data:
    print(row)

{'question': 'Tai nạn giao thông đường sắt nghiêm trọng là gì?', 'answer': 'Tai nạn giao thông đường sắt nghiêm trọng là tai nạn có 01 người chết hoặc có từ 06 đến 08 người bị thương hoặc gây thiệt hại về tài sản có giá trị từ 100 triệu đồng đến dưới 500 triệu đồng.', 'context': 'Trong Nghị định này, các từ ngữ dưới đây được hiểu như sau:\n1. Niên hạn sử dụng phương tiện giao thông đường sắt là khoảng thời gian được phép khai thác trên đường sắt của phương tiện, tính từ thời điểm phương tiện đóng mới được cấp giấy chứng nhận chất lượng, an toàn kỹ thuật và bảo vệ môi trường của tổ chức đăng kiểm hoặc chứng chỉ chất lượng của nhà sản xuất đến thời điểm phương tiện giao thông đường sắt không được phép khai thác trên đường sắt.\n2. Niên hạn sử dụng phương tiện giao thông đường sắt được phép nhập khẩu là khoảng thời gian tối đa mà phương tiện giao thông đường sắt đã được khai thác, sử dụng trước khi được phép nhập khẩu.\n3. Chốt gác là nơi có bố trí người được giao nhiệm vụ để thường trực 

### Promt để đánh giá faithfullness 

In [43]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os 
load_dotenv()
llm = ChatGroq(model_name="llama3-70b-8192", temperature=0,api_key= os.getenv('GROQ_API_KEY1')) ## Replace to real LLMs (Cohere / Groq / OpenAI)


In [84]:
from langchain import PromptTemplate, FewShotPromptTemplate


examples = [{
    "question": "Who was Albert Einstein and what is he best known for?",
    "answer": "He was a German-born theoretical physicist, widely acknowledged to be one of the greatest and most influential physicists of all time. He was best known for developing the theory of relativity, he also made important contributions to the development of the theory of quantum mechanics.",
    "sentences": "He was a German-born theoretical physicist, widely acknowledged to be one of the greatest and most influential physicists of all time",
    "analysis": '''
            Albert Einstein was a German-born theoretical physicist.,
            Albert Einstein is recognized as one of the greatest and most influential physicists of all time.,
'''
}]


# Định nghĩa mẫu cho mỗi ví dụ
example_template = """
Examples :\n
<Input>
Question: {question}
Answer: {answer}
Sentences:
{sentences}
</Input>\n\n
<Output>
Analysis: {analysis}
</Output>
"""

example_prompt = PromptTemplate(
    input_variables=["question", "answer", "sentences"],
    template=example_template
)

# Định nghĩa prompt chính
prefix = "Given a question, an answer, and sentences from the answer analyze the complexity of each sentence given under 'sentences' and break down each sentence into one or more fully understandable statements while also ensuring no pronouns are used in each statement. Format the outputs in JSON. Below is an example.Respone with exactly the same structure with examples. Rely in Vietnamese"

suffix = """
Question: {question}
Answer: {answer}
Sentences:
{sentences}

analysis:
"""

few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["question", "answer", "sentences"],
    example_separator="\n\n"
)

In [85]:
print(example_prompt.format(**examples[0]))


Examples :

<Input>
Question: Who was Albert Einstein and what is he best known for?
Answer: He was a German-born theoretical physicist, widely acknowledged to be one of the greatest and most influential physicists of all time. He was best known for developing the theory of relativity, he also made important contributions to the development of the theory of quantum mechanics.
Sentences:
He was a German-born theoretical physicist, widely acknowledged to be one of the greatest and most influential physicists of all time
</Input>


<Output>
Analysis: 
            Albert Einstein was a German-born theoretical physicist.,
            Albert Einstein is recognized as one of the greatest and most influential physicists of all time.,

</Output>



In [88]:
new_example =   {
        "question": "Dụng cụ trong ngành tài nguyên môi trường là gì?",
        "answer": "Dụng cụ trong ngành tài nguyên môi trường là loại tài sản không đủ tiêu chuẩn về tài sản cố định theo quy định hiện hành của nhà nước, mà người lao động sử dụng để tác động, biến đổi vật liệu thành sản phẩm. Ví dụ: kìm, búa, cờ lê, quần áo bảo hộ và các dụng cụ khác tương tự.",
        "sentences": "Trong Thông tư này các từ ngữ dưới đây được hiểu như sau:\n1. Định mức kinh tế - kỹ thuật (sau đây gọi tắt là định mức): Là mức hao phí cần thiết về lao động về nguyên, nhiên vật liệu, máy móc thiết bị, dụng cụ và phương tiện để hoàn thành một đơn vị sản phẩm (hoặc một khối lượng công việc nhất định), trong một điều kiện cụ thể của các hoạt động điều tra cơ bản trong các lĩnh vực thuộc phạm vi quản lý nhà nước của Bộ.\n2. Dụng cụ là loại tài sản không đủ tiêu chuẩn về tài sản cố định theo quy định hiện hành của nhà nước mà người lao động sử dụng để tác động, biến đổi vật liệu thành sản phẩm (kìm, búa, cờ lê, quần áo bảo hộ và các dụng cụ khác tương tự).\n3. Vật liệu là đầu vào trong một quá trình sản xuất hoặc thực hiện nhiệm vụ chuyên môn cụ thể, được người lao động tác động, biến đổi hoàn toàn để thành sản phẩm theo yêu cầu đặt ra.\n4. Máy móc thiết bị là công cụ lao động thuộc tài sản cố định hữu hình và tài sản cố định vô hình theo tiêu chuẩn quy định hiện hành của nhà nước về tài sản cố định (không bao gồm nhà xưởng và quyền sử dụng đất) mà người lao động sử dụng để tác động, biến đổi vật liệu thành sản phẩm."
    }

In [89]:
prompt = few_shot_prompt_template.format(
    question=new_example["question"],
    answer=new_example["answer"],
    sentences=new_example["sentences"]
)
# ai_msg = llm.invoke(prompt)
# ai_msg

print(prompt.format(**examples[0]))


Given a question, an answer, and sentences from the answer analyze the complexity of each sentence given under 'sentences' and break down each sentence into one or more fully understandable statements while also ensuring no pronouns are used in each statement. Format the outputs in JSON. Below is an example.Respone with exactly the same structure with examples. Rely in Vietnamese


Examples :

<Input>
Question: Who was Albert Einstein and what is he best known for?
Answer: He was a German-born theoretical physicist, widely acknowledged to be one of the greatest and most influential physicists of all time. He was best known for developing the theory of relativity, he also made important contributions to the development of the theory of quantum mechanics.
Sentences:
He was a German-born theoretical physicist, widely acknowledged to be one of the greatest and most influential physicists of all time
</Input>


<Output>
Analysis: 
            Albert Einstein was a German-born theoretical ph

In [90]:
out=llm.invoke(prompt)

In [91]:

out.content

'Here is the output in the same structure as the example:\n\n<Output>\nAnalysis: \n            Dụng cụ trong ngành tài nguyên môi trường là loại tài sản không đủ tiêu chuẩn về tài sản cố định theo quy định hiện hành của nhà nước.,\n            Dụng cụ trong ngành tài nguyên môi trường được người lao động sử dụng để tác động, biến đổi vật liệu thành sản phẩm.,\n            Ví dụ của dụng cụ trong ngành tài nguyên môi trường bao gồm kìm, búa, cờ lê, quần áo bảo hộ và các dụng cụ khác tương tự.,\n            Định mức kinh tế - kỹ thuật là mức hao phí cần thiết về lao động, nguyên, nhiên vật liệu, máy móc thiết bị, dụng cụ và phương tiện để hoàn thành một đơn vị sản phẩm trong một điều kiện cụ thể của các hoạt động điều tra cơ bản trong các lĩnh vực thuộc phạm vi quản lý nhà nước của Bộ.,\n            Định mức kinh tế - kỹ thuật được gọi tắt là định mức.,\n            Vật liệu là đầu vào trong một quá trình sản xuất hoặc thực hiện nhiệm vụ chuyên môn cụ thể được người lao động tác động, bi

### Chấm điểm cho faithfullness

In [35]:
from langchain import PromptTemplate, FewShotPromptTemplate



examples = [{
            "context":'John is a student at XYZ University. He is pursuing a degree in Computer Science. He is enrolled in several courses this semester, including Data Structures, Algorithms, and Database Management. John is a diligent student and spends a significant amount of time studying and completing assignments. He often stays late in the library to work on his projects.',
            "statements": '''
                John is majoring in Biology.,
                John is taking a course on Artificial Intelligence.,
                John is a dedicated student.,
                John has a part-time job.,
            ''',
            "answer": '''
                        statement1: John is majoring in Biology,
                        reason": John's major is explicitly mentioned as Computer Science. There is no information suggesting he is majoring in Biology.,
                        verdict: 0,

                        statement2: John is taking a course on Artificial Intelligence.,
                        reason: The context mentions the courses John is currently enrolled in, and Artificial Intelligence is not mentioned. Therefore, it cannot be deduced that John is taking a course on AI.,
                        verdict: 0,

                        statement3: John is a dedicated student.,
                        reason: "The context states that he spends a significant amount of time studying and completing assignments. Additionally, it mentions that he often stays late in the library to work on his projects, which implies dedication.,
                        verdict: 1,

                        statement4: John has a part-time job.,
                        reason: There is no information given in the context about John having a part-time job.,
                        verdict: 0,
'''

}]




example_template = """
Examples :\n
<Input>
context: {context}
statements: {statements}
</Input>\n\n
<Output>
answer: {answer}
</Output>
"""

example_prompt = PromptTemplate(
    input_variables=["context", "statements"],
    template=example_template
)

# Định nghĩa prompt chính
prefix = "Your task is to judge the faithfulness of a series of statements based on a given context. For each statement you must return verdict as 1 if the statement can be directly inferred based on the context or 0 if the statement can not be directly inferred based on the context. Format the outputs in JSON. Below is an example.Respone with exactly the same structure with examples. Rely in Vietnamese"

suffix = """
context: {context}
statements: {statements}

answer:
"""

few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["context", "statements"],
    example_separator="\n\n"
)

In [31]:
print(examples[0]['statements'])


                John is majoring in Biology.,
                John is taking a course on Artificial Intelligence.,
                John is a dedicated student.,
                John has a part-time job.,
            


In [36]:
print(example_prompt.format(**examples[0]))


Examples :

<Input>
context: John is a student at XYZ University. He is pursuing a degree in Computer Science. He is enrolled in several courses this semester, including Data Structures, Algorithms, and Database Management. John is a diligent student and spends a significant amount of time studying and completing assignments. He often stays late in the library to work on his projects.
statements: 
                John is majoring in Biology.,
                John is taking a course on Artificial Intelligence.,
                John is a dedicated student.,
                John has a part-time job.,
            
</Input>


<Output>
answer: 
                        statement1: John is majoring in Biology,
                        reason": John's major is explicitly mentioned as Computer Science. There is no information suggesting he is majoring in Biology.,
                        verdict: 0,

                        statement2: John is taking a course on Artificial Intelligence.,
       

In [37]:
new_example = {
            "context": """Nam là sinh viên tại Đại học ABC. Anh đang theo học ngành Công nghệ Thông tin. Trong học kỳ này, Nam đang học nhiều môn, bao gồm Cấu trúc Dữ liệu, Thuật toán, và Quản lý Cơ sở Dữ liệu. Nam là một sinh viên chăm chỉ và dành nhiều thời gian để học và hoàn thành các bài tập. Anh thường ở lại thư viện muộn để làm các dự án của mình.""",
            "statements": """
                Nam đang theo học ngành Y học.,
                Nam đang học môn Trí tuệ Nhân tạo.,
                Nam là một sinh viên chăm chỉ.,
                Nam có một công việc bán thời gian.,
            """,
            "answer": """
{
                        "statement": "Nam đang theo học ngành Y học.",
                        "reason": "Ngành học của Nam được nêu rõ là Công nghệ Thông tin. Không có thông tin nào cho thấy Nam đang theo học ngành Y học.",
                        "verdict": 0,

                        "statement": "Nam đang học môn Trí tuệ Nhân tạo.",
                        "reason": "Ngữ cảnh đề cập đến các môn học mà Nam đang theo học, và Trí tuệ Nhân tạo không được đề cập đến. Vì vậy, không thể suy luận rằng Nam đang học môn Trí tuệ Nhân tạo.",
                        "verdict": 0,

                        "statement": "Nam là một sinh viên chăm chỉ.",
                        "reason": "Ngữ cảnh nêu rằng Nam dành nhiều thời gian để học và hoàn thành các bài tập. Ngoài ra, việc Nam thường ở lại thư viện muộn để làm các dự án của mình ngụ ý rằng anh ấy rất chăm chỉ.",
                        "verdict": 1,

                        "statement": "Nam có một công việc bán thời gian.",
                        "reason": "Không có thông tin nào trong ngữ cảnh đề cập đến việc Nam có một công việc bán thời gian.",
                        "verdict": 0.
"""
}


In [38]:
result = " ".join(new_example["statements"])
print(type(result))

<class 'str'>


In [40]:
prompt = few_shot_prompt_template.format(
    context=new_example["context"],
    statements= new_example["statements"],
)

print(prompt.format(**examples[0]))


Your task is to judge the faithfulness of a series of statements based on a given context. For each statement you must return verdict as 1 if the statement can be directly inferred based on the context or 0 if the statement can not be directly inferred based on the context. Format the outputs in JSON. Below is an example.Respone with exactly the same structure with examples. Rely in Vietnamese


Examples :

<Input>
context: John is a student at XYZ University. He is pursuing a degree in Computer Science. He is enrolled in several courses this semester, including Data Structures, Algorithms, and Database Management. John is a diligent student and spends a significant amount of time studying and completing assignments. He often stays late in the library to work on his projects.
statements: 
                John is majoring in Biology.,
                John is taking a course on Artificial Intelligence.,
                John is a dedicated student.,
                John has a part-time jo

In [42]:
out=llm.invoke(prompt)
out.content

'Here is the output in the same structure as the example:\n\nanswer: \n                        statement1: Nam đang theo học ngành Y học.,\n                        reason: Ngành học của Nam được nêu rõ là Công nghệ Thông tin. Không có thông tin nào cho thấy anh đang theo học ngành Y học.,\n                        verdict: 0,\n\n                        statement2: Nam đang học môn Trí tuệ Nhân tạo.,\n                        reason: Văn cảnh nêu rõ các môn học mà Nam đang theo học, và Trí tuệ Nhân tạo không được nêu. Do đó, không thể suy luận rằng Nam đang học môn Trí tuệ Nhân tạo.,\n                        verdict: 0,\n\n                        statement3: Nam là một sinh viên chăm chỉ.,\n                        reason: Văn cảnh nêu rõ rằng Nam dành nhiều thời gian để học và hoàn thành các bài tập. Ngoài ra, nó cũng nêu rằng anh thường ở lại thư viện muộn để làm các dự án của mình, điều này ngụ ý chăm chỉ.,\n                        verdict: 1,\n\n                        statement4: Nam 